In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from sklearn.preprocessing import Normalizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [2]:
dimensoes = [
    'situacao_setor',
    'porcentagem_domicilios_sem_coleta_de_lixo',
    'porcentagem_domicilios_sem_ligacao_rede_de_agua',
    'porcentagem_domicilios_sem_banheiros',
    'porcentagem_domicilios_sem_esgoto_ou_fossa',
    # # 'porcentagem_domicilios_tipo_comodo',
    'porcentagem_domicilios_cedidos_outra_forma',
    'porcentagem_domicilios_em_outra_condicao_de_ocupacao',
    'numero_de_banheiros_por_habitante',
    'porcentagem_responsaveis_nao_alfabetizado',
    'porcentagem_responsaveis_nao_alfabetizado_com_menos_de_30_anos',
    'porcentagem_responsaveis_renda_ate_3_salarios',
    # # 'porcentagem_responsaveis_menos_8_anos_de_estudo',
    # # 'anos_medios_estudo_responsavel',
    'renda_media_responsavel',
    'numero_de_domicilios_particulares_permanentes',
    'numero_de_domicilios_improvisados',
    'numero_de_pessoas_residentes',
    'porcentagem_de_responsaveis_com_menos_30_anos',
    'numero_medio_pessoas',
    # 'area_do_setor',
    'densidade_populacional',
    'densidade_domicilios'
]

In [3]:
df = pd.read_csv('resultados/modelo.csv.zip')
df.fillna(0, inplace=True)
# rmsp = (df.rm == 20)
subnormal = (df.tipo_do_setor == 1)
df.loc[:, 'subnormal'] = 0
df.loc[subnormal, 'subnormal'] = 1
df.loc[:, 'processado'] = False

/tmp/ipykernel_90405/4035007722.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('resultados/modelo.csv.zip')


In [4]:
r_subnormal = df.loc[(subnormal), ['rm', 'subnormal']].groupby('rm').count().reset_index()
cidades_fora_rms = r_subnormal[(r_subnormal.subnormal < 20) | (r_subnormal.rm == 0.)].rm.to_list()
rms = r_subnormal[(r_subnormal.subnormal >= 20) & (r_subnormal.rm != 0.)].rm.to_list()

In [5]:
rms

[1.0,
 2.0,
 3.0,
 4.0,
 7.0,
 8.0,
 9.0,
 10.0,
 11.0,
 13.0,
 14.0,
 15.0,
 16.0,
 17.0,
 18.0,
 19.0,
 20.0,
 21.0,
 22.0,
 23.0,
 26.0,
 27.0,
 31.0,
 34.0,
 35.0,
 40.0,
 41.0]

In [6]:
def processa_modelos(filter, rm=None, uf=None):

    registro = {}

    # filter = (df.rm == rm)
    df_rm = df.loc[(filter)]
    df.loc[(filter), 'processado'] = True

    # print(df_rm.nome_rm.unique()[0])
    if rm:
        registro['nome_rm_ou_UF'] = df_rm.nome_rm.unique()[0]
    
    if uf:
        registro['nome_rm_ou_UF'] = uf
    # print(df_rm.subnormal.value_counts())

    registro['quantidade_de_municipios_processados'] = df_rm.cod_municipio.unique().shape[0]

    setores_normais = df_rm.loc[filter & (df_rm.loc[filter].subnormal == 0)].shape[0]
    registro['quantidade_original_de_setores_normais'] = setores_normais
    
    setores_subnormais = df_rm.loc[filter & (df_rm.loc[filter].subnormal == 1)].shape[0]
    registro['quantidade_original_de_setores_subnormais'] = setores_subnormais
    # print(setores_normais, setores_subnormais)

    samples = min(setores_normais, setores_subnormais * 2)
    # samples = 1

    # prepara amostras e lições
    x = df_rm.loc[(filter) & (subnormal)]
    x_ = df_rm.loc[(filter) & (~subnormal)].sample(n=samples, random_state=51)
    X = pd.concat([x, x_]).loc[(filter), dimensoes]
    y = pd.concat([x, x_]).loc[(filter), 'subnormal']
    # X = df_rm.loc[(filter), dimensoes]
    # y = df_rm.loc[(filter), 'subnormal']

    # Processa os modelos, LDA
    # print(y.value_counts())
    clf = LinearDiscriminantAnalysis(solver='lsqr')
    clf.fit(X, y)
    # print(np.unique(clf.predict(X), return_counts=True))
    predicao = clf.predict(df_rm.loc[(filter), dimensoes])
    try: 
        atribuidos_como_subnormais = np.unique(predicao, return_counts=True)[1][1]
    except:
        atribuidos_como_subnormais = 0
    registro['atribuidos_como_subnormais_LDA'] = atribuidos_como_subnormais
    # print(clf.score(X, y))
    registro['proporcao_atribuido_original_LDA'] = atribuidos_como_subnormais / setores_subnormais
    registro['score_LDA'] = clf.score(X, y)
    df.loc[(filter), 'subnormal_LDA'] = predicao

    # Processa model RandonForest
    clf = RandomForestClassifier(max_depth=10, random_state=0)
    clf.fit(X, y)
    # print(np.unique(clf.predict(X), return_counts=True))
    predicao = clf.predict(df_rm.loc[(filter), dimensoes])
    try:
        atribuidos_como_subnormais = np.unique(predicao, return_counts=True)[1][1]
    except:
        atribuidos_como_subnormais = 0    
    registro['atribuidos_como_subnormais_RandomForest'] = atribuidos_como_subnormais
    # print(clf.score(X, y))
    registro['proporcao_atribuido_original_RandomForest'] = atribuidos_como_subnormais / setores_subnormais
    registro['score_RandomForest'] = clf.score(X, y)
    df.loc[(filter), 'subnormal_RandomForest'] = predicao


    # # Processa modelo SVC
    # clf = LinearSVC(random_state=0, tol=1e-5)
    # clf.fit(Normalizer().fit_transform(X), y)
    # # print(np.unique(clf.predict(X), return_counts=True))
    # predicao = clf.predict(Normalizer().fit_transform(df_rm.loc[(filter), dimensoes]))
    # try:
    #     atribuidos_como_subnormais = np.unique(clf.predict(Normalizer().fit_transform(df_rm.loc[(filter), dimensoes])), return_counts=True)[1][1]
    # except:
    #     atribuidos_como_subnormais = 0    
    # registro['atribuidos_como_subnormais_SVC'] = atribuidos_como_subnormais
    # # print(clf.score(X, y))
    # registro['proporcao_atribuido_original_SVC'] = atribuidos_como_subnormais / setores_subnormais
    # registro['score_SVC'] = clf.score(Normalizer().fit_transform(X), y)
    # df.loc[(filter), 'subnormal_SVC'] = predicao

    # Salva as predicoes em atributos
    return registro

In [7]:
df[df.rm == 1.0].cod_municipio.unique().shape[0]

8

In [8]:
# Processa cada um dos municípios da sua Região MEtropolitana
result = [] 

for rm in rms:

    rm_filter = (df.rm == rm)
    registro = processa_modelos(rm_filter, rm=rm)
    # Salva as predicoes em atributos
    result.append(registro)

In [9]:
result

[{'nome_rm_ou_UF': 'RM Manaus ',
  'quantidade_de_municipios_processados': 8,
  'quantidade_original_de_setores_normais': 2509,
  'quantidade_original_de_setores_subnormais': 390,
  'atribuidos_como_subnormais_LDA': 549,
  'proporcao_atribuido_original_LDA': 1.4076923076923078,
  'score_LDA': 0.7803418803418803,
  'atribuidos_como_subnormais_RandomForest': 589,
  'proporcao_atribuido_original_RandomForest': 1.5102564102564102,
  'score_RandomForest': 0.9914529914529915},
 {'nome_rm_ou_UF': 'RM Belém',
  'quantidade_de_municipios_processados': 6,
  'quantidade_original_de_setores_normais': 995,
  'quantidade_original_de_setores_subnormais': 1057,
  'atribuidos_como_subnormais_LDA': 1269,
  'proporcao_atribuido_original_LDA': 1.2005676442762536,
  'score_LDA': 0.8079922027290448,
  'atribuidos_como_subnormais_RandomForest': 1163,
  'proporcao_atribuido_original_RandomForest': 1.1002838221381268,
  'score_RandomForest': 0.9395711500974658},
 {'nome_rm_ou_UF': 'RM Macapá',
  'quantidade_de

In [10]:
df.subnormal_LDA.value_counts()

subnormal_LDA
0.0    99887
1.0    20827
Name: count, dtype: int64

In [11]:
df.columns

Index(['Cod_setor', 'cod_grande_regiao', 'uf', 'nome_UF', 'rm', 'nome_rm',
       'meso_regiao', 'cod_municipio', 'nome_municipio', 'situacao_setor',
       'tipo_do_setor', 'numero_de_domicilios', 'numero_moradores',
       'area_do_setor', 'porcentagem_domicilios_sem_coleta_de_lixo',
       'porcentagem_domicilios_sem_ligacao_rede_de_agua',
       'porcentagem_domicilios_sem_banheiros',
       'porcentagem_domicilios_sem_esgoto_ou_fossa',
       'porcentagem_domicilios_cedidos_outra_forma',
       'porcentagem_domicilios_em_outra_condicao_de_ocupacao',
       'numero_de_banheiros_por_habitante',
       'porcentagem_responsaveis_nao_alfabetizado',
       'porcentagem_responsaveis_nao_alfabetizado_com_menos_de_30_anos',
       'porcentagem_responsaveis_renda_ate_3_salarios',
       'renda_media_responsavel',
       'numero_de_domicilios_particulares_permanentes',
       'numero_de_domicilios_improvisados', 'numero_de_pessoas_residentes',
       'porcentagem_de_responsaveis_com_menos_30

In [12]:
r_ufs = df[df.rm.isin(cidades_fora_rms)].loc[(subnormal), ['nome_UF', 'subnormal']].groupby('nome_UF').count().reset_index()

In [13]:
r_ufs

,nome_UF,subnormal
0,AC,33
1,AL,11
2,AM,94
3,AP,30
4,BA,39
5,CE,15
6,ES,112
7,GO,9
8,MG,94
9,MS,9


In [14]:
df_setores = df[df.rm.isin(cidades_fora_rms)].loc[:, ['nome_UF', 'subnormal', 'Cod_setor']].groupby('nome_UF').agg({'subnormal': 'sum', 'Cod_setor':'count'})

In [15]:
df_setores[df_setores.subnormal < 20]

,subnormal,Cod_setor
nome_UF,,
AL,11,2410
CE,15,8796
GO,9,7953
MA,0,6663
MS,9,4207
MT,0,4411
PB,0,2898
PI,0,3863
RN,5,2714


In [16]:
df_setores[df_setores.subnormal < 20].sum()

subnormal       57
Cod_setor    49014
dtype: int64

In [17]:
cidades_fora_rms

[0.0, 6.0, 12.0, 24.0, 28.0, 29.0, 36.0]

In [18]:
r_ufs[r_ufs.subnormal >= 20]

,nome_UF,subnormal
0,AC,33
2,AM,94
3,AP,30
4,BA,39
6,ES,112
8,MG,94
10,PA,127
11,PE,29
12,PR,53
13,RJ,571


In [19]:
r_ufs[r_ufs.subnormal < 20]

,nome_UF,subnormal
1,AL,11
5,CE,15
7,GO,9
9,MS,9
14,RN,5
16,RR,3
20,TO,5


In [20]:
# df[(df.rm.isin(cidades_fora_rms))]

In [21]:
# Processa cada uma das UFs
df_uf = df[df.rm.isin(cidades_fora_rms)]
ufs = r_ufs[r_ufs.subnormal >= 20].nome_UF.to_list()

for uf in ufs:

    uf_filter = (df.nome_UF == uf) & (df.rm.isin(cidades_fora_rms))
    print(uf)

    registro = processa_modelos(uf_filter, uf=uf)
    # Salva as predicoes em atributos
    result.append(registro)


AC


AM
AP
BA
ES
MG
PA
PE
PR
RJ
RO
RS
SC
SP2


In [22]:
gr_filter = (df.cod_grande_regiao == 4.) & (df.subnormal_LDA.isna())

In [23]:
## TODO
## Processar as cidades restantes por MacroRegião 
## Região 4 (Sul) não possui licoes suficientes de municípios com setores subnormais
for gr in [1., 2., 3., 5.]:
    gr_filter = (df.cod_grande_regiao == gr) & (~df.processado)
    print(gr)

    if len(df[gr_filter]) > 0:
        registro = processa_modelos(gr_filter, uf=f'Macro Região {str(gr)}')
        # Salva as predicoes em atributos
        result.append(registro)

1.0
2.0
3.0
5.0


In [24]:
df[df.cod_grande_regiao == 4.]

,Cod_setor,cod_grande_regiao,uf,nome_UF,rm,nome_rm,meso_regiao,cod_municipio,nome_municipio,situacao_setor,...,numero_de_domicilios_improvisados,numero_de_pessoas_residentes,porcentagem_de_responsaveis_com_menos_30_anos,numero_medio_pessoas,densidade_populacional,densidade_domicilios,subnormal,processado,subnormal_LDA,subnormal_RandomForest
151719,410010305000001,4.0,41,PR,0.0,Municípios não pertencentes a estrutura de RM,4104.0,4100103.0,ABATIÁ,1.0,...,1.0,844.0,0.120401,2.832215,844.0,298.0,0,True,0.0,0.0
151720,410010305000002,4.0,41,PR,0.0,Municípios não pertencentes a estrutura de RM,4104.0,4100103.0,ABATIÁ,1.0,...,0.0,881.0,0.062914,2.917219,881.0,302.0,0,True,0.0,0.0
151721,410010305000003,4.0,41,PR,0.0,Municípios não pertencentes a estrutura de RM,4104.0,4100103.0,ABATIÁ,1.0,...,0.0,1228.0,0.123116,3.085427,1228.0,398.0,0,True,0.0,0.0
151722,410010305000004,4.0,41,PR,0.0,Municípios não pertencentes a estrutura de RM,4104.0,4100103.0,ABATIÁ,1.0,...,1.0,898.0,0.143836,3.096552,898.0,290.0,0,True,0.0,0.0
151723,410010305000005,4.0,41,PR,0.0,Municípios não pertencentes a estrutura de RM,4104.0,4100103.0,ABATIÁ,1.0,...,2.0,1021.0,0.130435,3.190625,1021.0,320.0,0,True,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238621,421970516000005,4.0,42,SC,33.0,RM Chapecó,4201.0,4219705.0,XAXIM,2.0,...,0.0,16.0,0.000000,2.666667,16.0,6.0,0,False,NaN,NaN
238622,421985305000001,4.0,42,SC,0.0,Municípios não pertencentes a estrutura de RM,4203.0,4219853.0,ZORTÉA,1.0,...,1.0,877.0,0.177049,2.884868,877.0,304.0,0,True,0.0,0.0
238623,421985305000002,4.0,42,SC,0.0,Municípios não pertencentes a estrutura de RM,4203.0,4219853.0,ZORTÉA,1.0,...,2.0,984.0,0.170088,2.902655,984.0,339.0,0,True,0.0,0.0
238624,421985305000003,4.0,42,SC,0.0,Municípios não pertencentes a estrutura de RM,4203.0,4219853.0,ZORTÉA,1.0,...,1.0,465.0,0.162338,3.039216,465.0,153.0,0,True,0.0,0.0


In [25]:
pd.DataFrame(result).to_excel('resultados/geral_por_RM_UF.xlsx')

In [26]:
pd.DataFrame(result).quantidade_de_municipios_processados.sum()

5467

In [27]:
df.subnormal_LDA.value_counts()

subnormal_LDA
0.0    263833
1.0     42485
Name: count, dtype: int64

In [28]:
df.subnormal.value_counts()

subnormal
0    294305
1     15815
Name: count, dtype: int64

In [29]:
df.to_csv('resultados/modelo_preditivo.csv.zip', compression='zip')

In [30]:
df[df.subnormal_LDA.isna()]

,Cod_setor,cod_grande_regiao,uf,nome_UF,rm,nome_rm,meso_regiao,cod_municipio,nome_municipio,situacao_setor,...,numero_de_domicilios_improvisados,numero_de_pessoas_residentes,porcentagem_de_responsaveis_com_menos_30_anos,numero_medio_pessoas,densidade_populacional,densidade_domicilios,subnormal,processado,subnormal_LDA,subnormal_RandomForest
152135,410115005000001,4.0,41,PR,25.0,RM Maringá,4103.0,4101150.0,ÂNGULO,1.0,...,0.0,535.0,0.139785,2.876344,535.0,186.0,0,False,NaN,NaN
152136,410115005000002,4.0,41,PR,25.0,RM Maringá,4103.0,4101150.0,ÂNGULO,1.0,...,0.0,513.0,0.113095,3.053571,513.0,168.0,0,False,NaN,NaN
152137,410115005000003,4.0,41,PR,25.0,RM Maringá,4103.0,4101150.0,ÂNGULO,5.0,...,1.0,167.0,0.178571,3.036364,167.0,55.0,0,False,NaN,NaN
152138,410115005000004,4.0,41,PR,25.0,RM Maringá,4103.0,4101150.0,ÂNGULO,8.0,...,0.0,266.0,0.100000,3.325000,266.0,80.0,0,False,NaN,NaN
152139,410115005000005,4.0,41,PR,25.0,RM Maringá,4103.0,4101150.0,ÂNGULO,8.0,...,0.0,168.0,0.148148,3.111111,168.0,54.0,0,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310115,170210905000098,0.0,0,TO,0.0,0,0.0,0.0,0,0.0,...,0.0,0.0,0.278552,0.000000,0.0,0.0,0,False,NaN,NaN
310116,170210905000099,0.0,0,TO,0.0,0,0.0,0.0,0,0.0,...,1.0,0.0,0.239437,0.000000,0.0,0.0,0,False,NaN,NaN
310117,170210905000100,0.0,0,TO,0.0,0,0.0,0.0,0,0.0,...,0.0,0.0,0.230769,0.000000,0.0,0.0,0,False,NaN,NaN
310118,170210905000101,0.0,0,TO,0.0,0,0.0,0.0,0,0.0,...,0.0,0.0,0.362264,0.000000,0.0,0.0,0,False,NaN,NaN


In [31]:
df

,Cod_setor,cod_grande_regiao,uf,nome_UF,rm,nome_rm,meso_regiao,cod_municipio,nome_municipio,situacao_setor,...,numero_de_domicilios_improvisados,numero_de_pessoas_residentes,porcentagem_de_responsaveis_com_menos_30_anos,numero_medio_pessoas,densidade_populacional,densidade_domicilios,subnormal,processado,subnormal_LDA,subnormal_RandomForest
0,120001305000001,1.0,12,AC,0.0,Municípios não pertencentes a estrutura de RM,1202.0,1200013.0,ACRELÂNDIA,1.0,...,0.0,957.0,0.159574,3.393617,957.0,282.0,0,True,0.0,0.0
1,120001305000002,1.0,12,AC,0.0,Municípios não pertencentes a estrutura de RM,1202.0,1200013.0,ACRELÂNDIA,1.0,...,2.0,1203.0,0.244186,3.517544,1203.0,342.0,0,True,0.0,0.0
2,120001305000003,1.0,12,AC,0.0,Municípios não pertencentes a estrutura de RM,1202.0,1200013.0,ACRELÂNDIA,1.0,...,1.0,1700.0,0.293996,3.526971,1700.0,482.0,0,True,1.0,1.0
3,120001305000004,1.0,12,AC,0.0,Municípios não pertencentes a estrutura de RM,1202.0,1200013.0,ACRELÂNDIA,5.0,...,0.0,182.0,0.226415,3.433962,182.0,53.0,0,True,0.0,0.0
4,120001305000005,1.0,12,AC,0.0,Municípios não pertencentes a estrutura de RM,1202.0,1200013.0,ACRELÂNDIA,8.0,...,0.0,305.0,0.197917,3.177083,305.0,96.0,0,True,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310115,170210905000098,0.0,0,TO,0.0,0,0.0,0.0,0,0.0,...,0.0,0.0,0.278552,0.000000,0.0,0.0,0,False,NaN,NaN
310116,170210905000099,0.0,0,TO,0.0,0,0.0,0.0,0,0.0,...,1.0,0.0,0.239437,0.000000,0.0,0.0,0,False,NaN,NaN
310117,170210905000100,0.0,0,TO,0.0,0,0.0,0.0,0,0.0,...,0.0,0.0,0.230769,0.000000,0.0,0.0,0,False,NaN,NaN
310118,170210905000101,0.0,0,TO,0.0,0,0.0,0.0,0,0.0,...,0.0,0.0,0.362264,0.000000,0.0,0.0,0,False,NaN,NaN


In [32]:
df.processado.value_counts()

processado
True     306318
False      3802
Name: count, dtype: int64